In [1]:
import os
import pickle

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

def save_labels(arr, filename):
    pd_array = pd.DataFrame(arr)
    pd_array.index.names = ["Id"]
    pd_array.columns = ["Prediction"]
    pd_array.to_csv(filename)

def load_labels(filename):
    return pd.read_csv(filename, index_col=0).values.ravel()

In [68]:
def trans(lis):
    return np.array([  [ 1.0*(x == i) for i in xrange(10)] for x in lis  ])

X_train = np.load("E:\\Desktop\\dane_ml\\X_train.npy")
y_train = load_labels("E:\\Desktop\\dane_ml\\y_train.csv")
X_test = np.load("E:\\Desktop\\dane_ml\\X_test.npy")

# Mój keras ma kolorki na końcu
X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)
input_shape = (32, 32, 1)


In [54]:
#if not os.path.isfile("baseline.pkl"):
#    lr = Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression(verbose=43))])
#    lr.fit(X_train, y_train)
#    print("Train acc:", accuracy_score(y_train, lr.predict(X_train)))
#    with open("baseline.pkl", 'w') as f_out:
#        pickle.dump(lr, f_out)
#else:
#    with open("baseline.pkl", 'r') as f_in:
#        lr = pickle.load(f_in)
#
#save_labels(model.predict(X_test), "y_pred.csv")

# CZĘŚĆ PIERWSZA

xgboost?

In [74]:
from sklearn.model_selection import train_test_split
first_X_train, first_X_test, first_y_train, first_y_test = train_test_split(X_train[:1200], y_train[:1200], test_size=0.33, random_state=43)
first_y_train = trans(first_y_train)
first_y_test = trans(first_y_test)

input_shape=first_X_train[0].shape
print input_shape


(32L, 32L, 3L)


In [75]:
print first_y_test[3]

[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]


In [115]:
os.environ['KERAS_BACKEND'] = 'theano'

from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Conv2D, MaxPooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))

model.add(Conv2D(256, (3, 3), activation='relu'))


model.add(Flatten())  
model.add(Dense(10))
model.add(Activation('sigmoid'))

model.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(),
              metrics=['accuracy'])

In [ ]:
model.fit(first_X_train, first_y_train,
          batch_size=50,
          epochs=3,
          verbose=1,
          validation_data=(first_X_test, first_y_test))


Train on 804 samples, validate on 396 samples
Epoch 1/3
200/804 [======>.......................] - ETA: 12s - loss: 8.2363 - acc: 0.1000

In [99]:
score = model.evaluate(first_X_test, first_y_test, verbose=0)
print score

[14.490005156006475, 0.10101010116061779]


In [110]:
print np.around(model.predict(first_X_test)[0], 3)

print first_y_test[0]

[ 0.  1.  1.  1.  0.  1.  1.  0.  1.  1.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
